# Introdução

In [ ]:
import yfinance as yf
import pandas as pd

import pandas as pd 
import numpy as np 
import math
import matplotlib. pyplot as plt 
import seaborn as sns 
from scipy import stats


In [ ]:
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_columns', None)

In [ ]:
tickers = ['WEGE3.SA','AURE3.SA','LEVE3.sa','CSMG3.SA'
           ,'PETR4.SA','PETR3.SA','MTRE3.SA','BRAP4.SA','BRAP3.SA'
           ,'GOAU4.SA','AGRO3.SA','SLCE3.SA'
           ,'VALE3.SA','CPFE3.SA','SBSP3.SA'
           ,'ITUB3.SA','ITUB4.SA','BMGB4.SA']

stocks = yf.download(tickers,period='2y', interval='1d')

df = pd.DataFrame(stocks['Adj Close']).reset_index()

df.set_index('Date')

#df.rename(columns={'AMER3.SA':'AMERICANAS','MGLU3.SA':'MAGALU'}, inplace=True)

#df.columns.droplevel(1)

df.tail()

In [ ]:

dataset = df.copy()
dataset.drop(labels=['Date'], axis=1, inplace=True)
taxas_retorno = (dataset/dataset.shift(1))-1
taxas_retorno.std()*100

In [ ]:
taxas_retorno.std()*252


In [ ]:
taxas_retorno.cov()

In [ ]:
taxas_retorno.corr()

In [ ]:
#taxas_retorno_ame_cvc = taxas_retorno[['AMER3.SA','CVCB3.SA']]

In [ ]:
#taxas_retorno_ame_cvc.cov()*100

In [ ]:
pesos = np.array([0.5, 0.5])

In [ ]:
#np.dot(pesos,taxas_retorno_ame_cvc.cov()*246,pesos).sum()


In [ ]:
# math.sqrt(np.dot(pesos,taxas_retorno_ame_cvc.cov()*246,pesos).sum())


In [ ]:
# pesos = np.array([0.2, 0.2, 0.2, 0.2, 0.1, 0.1])

In [ ]:
# taxas_retorno.cov()*246

In [ ]:
# np.dot(taxas_retorno.cov()*246,pesos)

In [ ]:
# np.dot(pesos,taxas_retorno.cov()*246,pesos).sum()

In [ ]:
#math.sqrt(np.dot(pesos,taxas_retorno.cov()*246,pesos).sum())

# Analise Técnica

In [ ]:
ticker = tickers[0]  # Exemplo com a Wege
stock = yf.download(ticker,period='2y', interval='1d')

stock.head()



In [ ]:
stock = yf.Ticker(tickers[0])
df = stock.history(period="5y")
df['Close'].plot(title="{}'s stock price".format(tickers[0]))

In [ ]:
dir(stock)

In [ ]:
for k,v in stock.info.items():
    print (k, '\t', v, '\n')

In [ ]:
stock.recommendations

In [ ]:
# stock.sustainability

In [ ]:
stock.actions

In [ ]:
stock.dividends

In [ ]:
stock.info

In [ ]:
stock.info['dividendYield']

In [ ]:
stock.info['beta']

In [ ]:
list(map(lambda x: stock.info[x], 
         ['targetLowPrice', 'targetMeanPrice','targetMedianPrice','targetHighPrice']))

In [ ]:
stock.info['industry']

In [ ]:
stock.major_holders

In [ ]:
stock.institutional_holders

In [ ]:

# Obter histórico de preços
hist = stock.history(period="5y")
hist.tail()

In [ ]:
stock.get_cashflow()

In [ ]:
pd.DataFrame(stock.calendar)


In [ ]:
financials = stock.financials

financials

In [ ]:
stock.cashflow

In [ ]:
stock.cash_flow

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

def fetch_earnings_data(ticker):
    # Set up Selenium to run headlessly
    options = Options()
    options.headless = True
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920x1080")

    driver = webdriver.Chrome(options=options)
    url = f"https://finance.yahoo.com/calendar/earnings?symbol={ticker}"
    driver.get(url)

    # Find the rows of the earnings table
    rows = driver.find_elements(By.CSS_SELECTOR, 'table tbody tr')

    data = []

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, 'td')
        cols = [elem.text for elem in cols]
        data.append(cols)

    # Close the WebDriver
    driver.quit()

    # Assuming the data structure is as expected, create a DataFrame
    columns = ['Symbol', 'Company', 'Earnings Date', 'EPS Estimate', 'Reported EPS', 'Surprise(%)']
    df = pd.DataFrame(data, columns=columns)

    return df

# Example usage:
ticker = tickers[0]
earnings_data = fetch_earnings_data(ticker)

# Extract the time and timezone information into a new column
earnings_data['Earnings Time'] = earnings_data['Earnings Date']

earnings_data['EPS Estimate'] = earnings_data['EPS Estimate'].replace('+', '').replace('-', None).astype(float)
earnings_data['Reported EPS'] = earnings_data['Reported EPS'].replace('+', '').replace('-', None).astype(float)


# Extract just the date part from the "Earnings Date" column
earnings_data['Earnings Date'] = earnings_data['Earnings Date'].str.extract(r'(\b\w+ \d{1,2}, \d{4})')

# Convert string date to datetime
earnings_data['Earnings Date'] = pd.to_datetime(earnings_data['Earnings Date'])

earnings_data['Surprise(%)'] = earnings_data['Surprise(%)'].str.replace('+', '').replace('-', None).astype(float)

earnings_data

In [ ]:
earnings = earnings_data.dropna().sort_values(by='Earnings Date',ascending=False)

earnings

## P/L

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np



# Obtenha o preço de fechamento mais recente
latest_price = hist['Close'].iloc[-1]

# Obtenha o lucro por ação (Earnings Per Share - EPS)
# Aqui estamos assumindo que os dados de lucros (earnings) estão em milhares
eps = earnings['Reported EPS'].iloc[0]
print(f"Latest closing price per share for {ticker}: ${latest_price:.4f}")
print(f"Earnings per share (EPS) for {ticker}: ${eps:.2f}")



In [ ]:
# Calcule o P/L
pl_ratio = latest_price / eps

print(f"P/L ratio for {ticker}: {pl_ratio:.2f}")

In [ ]:
balance_sheet = stock.balance_sheet

balance_sheet

In [ ]:
closing_prices = hist['Close']
closing_prices

In [ ]:
balance_sheet.loc['Stockholders Equity', :]

In [ ]:
financials.loc['Net Income', :]

In [ ]:
financials.head()

In [ ]:
from datetime import datetime

dt = financials.columns[0]
dt = datetime.strftime(dt, "%Y-%m-%d")

dt

In [ ]:
type(dt)

In [ ]:
closing_prices = pd.DataFrame(hist['Close']).reset_index()

closing_prices['DateStr'] = pd.to_datetime(closing_prices['Date']).dt.date

closing_prices['Year'] = pd.to_datetime(closing_prices['Date']).dt.year


closing_prices.sort_values(by='Date', ascending=False, inplace=True)

closing_prices['Date'] = pd.to_datetime(closing_prices['DateStr'])

closing_prices.head()




In [ ]:
closing_prices.info()

In [ ]:
closing_prices.tail()

In [ ]:
last_prices = pd.DataFrame(closing_prices.groupby(by='Year')['Date'].max())
last_prices

In [ ]:
prices = pd.merge(closing_prices , last_prices).sort_values(by='Date',ascending=False)

In [ ]:
closing_price = closing_prices[(closing_prices['Date'] == '2023-12-28')]['Close']

In [ ]:
prices

In [ ]:
prices1 = prices.loc[1:]
prices1

In [ ]:

# Exemplo de cálculo de P/L com dados do Yahoo Finance
df_eps = pd.DataFrame(financials.loc['Net Income', :] / balance_sheet.loc['Stockholders Equity', :]).reset_index()

df_eps

In [ ]:
financials.loc['Basic EPS', :]

In [ ]:

df_eps.columns = ['Date','eps']



In [ ]:
df_eps['Year'] = pd.to_datetime(df_eps['Date']).dt.year

In [ ]:
df_eps

In [ ]:
type(df_eps)

In [ ]:
prices

In [ ]:
df_eps

In [ ]:
eps_prices = pd.merge(df_eps , prices, on='Year')
eps_prices

In [ ]:
prices

#balance_sheet

In [ ]:
df_pl_ratio = eps_prices.copy()
df_pl_ratio

In [ ]:
df_pl_ratio['pl_ratio'] = df_pl_ratio['Close']/df_pl_ratio['eps']

In [ ]:
df_pl_ratio

In [ ]:
# prices/eps

In [ ]:
#
# stock.info


Calculando P/L (Preço/Lucro)
df['P/L'] = df['price_per_share'] / df['earnings_per_share']

Calculando P/VP (Preço/Valor Patrimonial)
df['P/VP'] = df['price_per_share'] / df['book_value_per_share']

Calculando P/S (Preço/Vendas)
df['P/S'] = df['price_per_share'] / df['sales_per_share']

Calculando PEG (Price to Earnings Growth)
df['PEG'] = df['P/L'] / df['growth_rate']

Calculando ROE (Retorno sobre o Patrimônio Líquido)
df['ROE'] = df['earnings_per_share'] / df['book_value_per_share']

